In [1]:
import pandas as pd
import json
from ast import literal_eval
from pandas.io.json import json_normalize
from datetime import datetime
import re
import requests
import numpy as np

In [2]:
#Pull website data and extract table
url = 'https://www.the-numbers.com/movies/franchises'

response = requests.get(url)

tables = pd.read_html(response.text)

f_all_data_df = tables[0]

In [3]:
#Keep relevant rows: Franchise name and Worldwide Box Office
franchise_df = f_all_data_df[["Franchise", "Worldwide Box Office"]]

In [4]:
#Remove currency formatting and drop franchises with no value or 0  
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace(',', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace('$', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].astype(float)

franchise_df.replace(0, np.nan, inplace=True)
franchise_df.dropna(inplace=True)

C:\Users\bvera\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\bvera\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bvera\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [5]:
#Export franchise data
#franchise_df.to_csv('franchise_df.csv', index=False)

In [15]:
#Read metadata CSV
csv_file = "movies_metadata.csv"

metadata_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape', low_memory=False)

In [16]:
#Keep relevant rows: original_title, collection, revenue. Drop rows with 0 value
franchise_single = metadata_df[['original_title', 'belongs_to_collection', 'revenue']]

franchise_single.astype({'revenue': 'float64'}).dtypes

franchise_single = franchise_single.query("revenue > 0")
franchise_single = franchise_single.reset_index(drop=True)
franchise_single.dropna(inplace=True)

In [ ]:
#Export data for individual movies
#franchise_single.to_csv('metadata_franchise.csv', index=False)

In [12]:
#Create empty dataframe for final table
final_table = pd.DataFrame (columns = ['Franchise', 'Movie Title','Movie Revenue', 
                                       'Total Franchise Revenue', '% of Total Franchise Revenue'])

In [13]:
#Match if collection contains franchise name and input desired data into final table
for idx, fn in franchise_df.iterrows():
    search = fn["Franchise"]
    wwbo = fn["Worldwide Box Office"]
    title_match = franchise_single["belongs_to_collection"].str.contains(search)
    indexes = (title_match[title_match==True].index.values)
    
    revenue = franchise_single.loc[indexes, 'revenue'] 
    movie_title = franchise_single.loc[indexes, 'original_title'] 
    franchise_performance = (revenue / wwbo)*100

    final_data = {'Franchise': search, 'Movie Title': movie_title, 'Movie Revenue': revenue, 
                  'Total Franchise Revenue': wwbo, '% of Total Franchise Revenue': franchise_performance}
    final_table = final_table.append(pd.DataFrame(final_data, columns= ['Franchise', 'Movie Title', 
                                                                        'Movie Revenue', 'Total Franchise Revenue', 
                                                                        '% of Total Franchise Revenue']),ignore_index=True)

C:\Users\bvera\Anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [14]:
#Clean final table
delete='[REC]'

clean_final = final_table[final_table.Franchise != delete]

#Export final table to CSV
clean_final.to_csv('Franchise_Performance_By_Movie.csv', index=False)

In [17]:
csv_file = "movies_metadata.csv"

ratings_data = "movie_ratings.csv"


# df = pd.read_csv(csv_file,'unicode_escape')

movies_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape')

ratings_df = pd.read_csv(ratings_data)

# csv_file.encode('utf-8').strip(csv_file)

C:\Users\bvera\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
# df.head()

In [19]:
movies_df["genres"].head(1)

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
Name: genres, dtype: object

In [20]:
 # Step 2: Isolation of columns that are relevant
movie_genres = movies_df[['id', 'genres']]

In [21]:
# Step 3: Iterate over each row from our isolated dataset
# to_string converts the movie_genre dataframe to a string. movie_genre is a variable prior.
parse_genre_df = pd.DataFrame()

for row_number, movie in movie_genres.iterrows():
    genre_list = []
#     print(type(movie_genres))
#     print(type(movie_genres.to_string()))
    for genre_counter, genre in enumerate(literal_eval(movie['genres'])):
#   print(movie['id'], genre.get('name'))
        genre_list.append(genre.get('name'))
            
        

    # Associate the parsed genres with the movie id
    parse_genre_df = parse_genre_df.append({"id": movie['id'],
                                            'genre': ', '.join(map(str, genre_list))},
                                           ignore_index=True,
                                           sort=False)
   

In [22]:
# parse_genre_df.head()

In [23]:
prod_company = movies_df[["id", "production_companies"]]

In [24]:
parse_companies_df = pd.DataFrame()

for row_number, movie in prod_company.iterrows():
    company_list = []
    try:
        for company_counter, company in enumerate(literal_eval(movie['production_companies'])):
            company_list.append(company.get('name'))
    except ValueError:
        pass
        
    parse_companies_df = parse_companies_df.append({'id': movie['id'],
                                                   'production_companies': ', '.join(map(str, company_list))},
                                           ignore_index=True,
                                           sort=False)


In [25]:
# parse_companies_df.head()

In [26]:
movies_df.drop(['genres', 'production_companies'], axis=1, inplace = True)

In [27]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [28]:
movies_df = movies_df.merge(parse_companies_df, on="id")

In [29]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'production_companies'],
      dtype='object')

In [30]:
movies_df = movies_df.merge(parse_genre_df, on="id")

In [31]:
movies_df. columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'production_companies', 'genre'],
      dtype='object')

In [40]:
movies_df.head(5)


,budget,id,imdb_id,release_date,revenue,title,vote_average,production_companies,genre
0,30000000,862,tt0114709,10/30/1995,373554033.0,Toy Story,7.7,Pixar Animation Studios,"Animation, Comedy, Family"
1,65000000,8844,tt0113497,12/15/1995,262797249.0,Jumanji,6.9,"TriStar Pictures, Teitler Film, Interscope Com...","Adventure, Fantasy, Family"
2,0,15602,tt0113228,12/22/1995,0.0,Grumpier Old Men,6.5,"Warner Bros., Lancaster Gate","Romance, Comedy"
3,16000000,31357,tt0114885,12/22/1995,81452156.0,Waiting to Exhale,6.1,Twentieth Century Fox Film Corporation,"Comedy, Drama, Romance"
4,0,11862,tt0113041,2/10/1995,76578911.0,Father of the Bride Part II,5.7,"Sandollar Productions, Touchstone Pictures",Comedy


In [35]:
movies_df.to_csv('new_movie_metadata.csv', index=False)

In [38]:
movies_df = movies_df.drop(['adult', 'belongs_to_collection', 'genres', 'homepage', 'original_language',
                'original_title', 'overview', 'popularity', 'poster_path', 'production_countries',
                'runtime', 'spoken_languages', 'status', 'tagline', 'video', 'vote_count'], axis=1)

KeyError: "['adult' 'belongs_to_collection' 'genres' 'homepage' 'original_language'\n 'original_title' 'overview' 'popularity' 'poster_path'\n 'production_countries' 'runtime' 'spoken_languages' 'status' 'tagline'\n 'video' 'vote_count'] not found in axis"

In [39]:
movies_df.head()

,budget,id,imdb_id,release_date,revenue,title,vote_average,production_companies,genre
0,30000000,862,tt0114709,10/30/1995,373554033.0,Toy Story,7.7,Pixar Animation Studios,"Animation, Comedy, Family"
1,65000000,8844,tt0113497,12/15/1995,262797249.0,Jumanji,6.9,"TriStar Pictures, Teitler Film, Interscope Com...","Adventure, Fantasy, Family"
2,0,15602,tt0113228,12/22/1995,0.0,Grumpier Old Men,6.5,"Warner Bros., Lancaster Gate","Romance, Comedy"
3,16000000,31357,tt0114885,12/22/1995,81452156.0,Waiting to Exhale,6.1,Twentieth Century Fox Film Corporation,"Comedy, Drama, Romance"
4,0,11862,tt0113041,2/10/1995,76578911.0,Father of the Bride Part II,5.7,"Sandollar Productions, Touchstone Pictures",Comedy


In [41]:
#renaming the columns
movies_df = movies_df.rename(columns={'budget': 'production_budget', 'id': 'movie_id', 'title': 'movie_name', 
                                      'vote_average': 'imdb_rating'})

In [42]:
# Drop all blank movie ID's

movies_df = movies_df.dropna(subset=['movie_id'])

In [43]:
# Removing the 0's before movie_id 

movies_df['movie_id'] = movies_df['movie_id'].astype(str).str.replace('\D+', '')

In [44]:
# Setting movie_id to int

movies_df['movie_id'] = movies_df['movie_id'].apply(lambda x: int(x))

In [45]:
#working on the RATING CSV here
ratings_df = ratings_df.groupby(['movieId'], as_index = False).mean()

In [46]:
# Add rating to movies_df

movies_df['rating'] = movies_df.movie_id.map(ratings_df.set_index('movieId')['rating'].to_dict())

In [47]:
movies_df.head()

,production_budget,movie_id,imdb_id,release_date,revenue,movie_name,imdb_rating,production_companies,genre,rating
0,30000000,862,tt0114709,10/30/1995,373554033.0,Toy Story,7.7,Pixar Animation Studios,"Animation, Comedy, Family",3.454545
1,65000000,8844,tt0113497,12/15/1995,262797249.0,Jumanji,6.9,"TriStar Pictures, Teitler Film, Interscope Com...","Adventure, Fantasy, Family",4.125000
2,0,15602,tt0113228,12/22/1995,0.0,Grumpier Old Men,6.5,"Warner Bros., Lancaster Gate","Romance, Comedy",NaN
3,16000000,31357,tt0114885,12/22/1995,81452156.0,Waiting to Exhale,6.1,Twentieth Century Fox Film Corporation,"Comedy, Drama, Romance",NaN
4,0,11862,tt0113041,2/10/1995,76578911.0,Father of the Bride Part II,5.7,"Sandollar Productions, Touchstone Pictures",Comedy,NaN


In [48]:
#Yearly Production Studio code here

In [49]:
release_date = movies_df[['movie_id','release_date']].head()

In [50]:
#a for loop to pick up dates in the excel that are messy.
year_list = []

for row_number, date in movies_df["release_date"].items():
    if isinstance(date, float) or len(date) <=3:
        year_list.append(None)
    else:
        try:
            date_year = datetime.strptime(date, "%m/%d/%Y")
            year_list.append(date_year.year)
        except ValueError:
            date_year = datetime.strptime(date, "%Y-%m-%d")
            year_list.append(date_year.year)

In [51]:
#new column, who dis?
movies_df["release_year"] = year_list

In [52]:
movies_df[["release_date","release_year"]]

,release_date,release_year
0,10/30/1995,1995.0
1,12/15/1995,1995.0
2,12/22/1995,1995.0
3,12/22/1995,1995.0
4,2/10/1995,1995.0
...,...,...
45653,NaN,NaN
45654,11/17/2011,2011.0
45655,8/1/2003,2003.0
45656,10/21/1917,1917.0


In [53]:
production = movies_df[["production_companies","revenue","production_budget","movie_name","release_year"]].copy()

In [54]:
production = production.dropna(subset=['movie_name','production_companies'])

In [55]:
top = production.sort_values(by = ["revenue"],ascending = False)

In [56]:
#filtering production companies with Marvel in string
Marvel = production[production['production_companies'].str.contains('Marvel')]

In [57]:
#looking at the most recent film in the table
sort_year = Marvel.sort_values(by = ["release_year"],ascending = False)

In [58]:
#sorting by title since this table didn't have an index prior. dropped films that had 0 info
top_ten = sort_year.set_index('movie_name').drop(['Team Thor','Marvel Studios: Assembling a Universe',
                                             'Marvel One-Shot: All Hail the King',
                                             'Iron Man & Captain America: Heroes United',
                                             'Marvel: 75 Years, From Pulp to Pop!','Iron Man & Hulk: Heroes United',
                                             'Marvel One-Shot: Agent Carter','Marvel One-Shot: Item 47',
                                             'Thor: Tales of Asgard','Planet Hulk'])

In [59]:
#reseting the index
top_ten.reset_index(level='movie_name')

,movie_name,production_companies,revenue,production_budget,release_year
0,Logan,"Twentieth Century Fox Film Corporation, Donner...",6.168018e+08,97000000,2017.0
1,Guardians of the Galaxy Vol. 2,"Walt Disney Pictures, Marvel Studios",8.634161e+08,200000000,2017.0
2,Thor: Ragnarok,"Walt Disney Pictures, Marvel Studios",0.000000e+00,0,2017.0
3,Deadpool,"Twentieth Century Fox Film Corporation, Marvel...",7.831130e+08,58000000,2016.0
4,Captain America: Civil War,"Studio Babelsberg, Marvel Studios, Walt Disney...",1.153304e+09,250000000,2016.0
5,Doctor Strange,"Walt Disney Pictures, Marvel Studios",6.777184e+08,165000000,2016.0
6,X-Men: Apocalypse,"Twentieth Century Fox Film Corporation, Donner...",5.439348e+08,178000000,2016.0
7,Fantastic Four,"Twentieth Century Fox Film Corporation, Marv F...",1.679776e+08,120000000,2015.0
8,Avengers: Age of Ultron,"Marvel Studios, Prime Focus, Revolution Sun St...",1.405404e+09,280000000,2015.0
9,Ant-Man,Marvel Studios,5.193120e+08,130000000,2015.0


In [60]:
#convert budget to float since sum below wouldnt work as a object
top_ten["production_budget"] = top_ten["production_budget"].astype(float)

In [61]:
revenue_sum = top_ten.groupby(['release_year'])['revenue'].sum()

budget_sum = top_ten.groupby(['release_year'])['production_budget'].sum()

In [62]:
merge_inner = pd.merge(left=revenue_sum, right=budget_sum, left_on='release_year', right_on='release_year')

In [63]:
#final table with total revenue and budget per year
merge_inner.sort_values(by = ["release_year"],ascending = False)

,revenue,production_budget
release_year,,
2017.0,1.480218e+09,297000000.0
2016.0,3.158071e+09,651000000.0
2015.0,2.092693e+09,530000000.0
2014.0,2.941675e+09,790000000.0
2013.0,2.275452e+09,490000000.0
2012.0,2.271774e+09,435000000.0
2011.0,1.322738e+09,508000000.0
2010.0,6.239333e+08,200000000.0
2009.0,3.730629e+08,150000000.0
